In [71]:
import os
import pandas as pd
import numpy as np
from keras.applications.resnet_v2 import ResNet50V2
import tensorflow as tf
from tensorflow.keras import models, Sequential, layers, optimizers, callbacks
from tensorflow.keras.layers import Dense, Reshape
from keras.preprocessing import image 
from tqdm import tqdm
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

In [30]:
cat0 = 'n02085936' #Maltese_dog
cat1 = 'n02088364' #beagle
cat2 = 'n02090721' # Irish_wolfhound
cat3 = 'n02092339' # Weimaraner
cat4 = 'n02093754' # Border_terrier
cat5 = 'n02094433' # Yorkshire_terrier
cat6 = 'n02099601' # golden_retriever
cat7 = 'n02100583' # vizsla
cat8 = 'n02105641' # Old_English_sheepdog
cat9 = 'n02106550' # Rottweiler
cat10 = 'n02106662' # German_shepherd
cat11 = 'n02108089' # boxer
cat12 = 'n02108422' # bull_mastiff
cat13 = 'n02112137' # chow
cat14 = 'n02110185' # Siberian_husky
all_cat = [cat0, cat1, cat2, cat3, cat4, cat5, cat6, cat7, cat8, cat9, cat10, cat11, cat12, cat13, cat14]

In [35]:
def categorise(x):
    if x[:9] in cat0: return 0
    elif x[:9] in cat1: return 1
    elif x[:9] in cat2: return 2
    elif x[:9] in cat3: return 3
    elif x[:9] in cat4: return 4    
    elif x[:9] in cat5: return 5    
    elif x[:9] in cat6: return 6
    elif x[:9] in cat7: return 7
    elif x[:9] in cat8: return 8
    elif x[:9] in cat9: return 9
    elif x[:9] in cat10: return 10    
    elif x[:9] in cat11: return 11   
    elif x[:9] in cat12: return 12 
    elif x[:9] in cat13: return 13
    elif x[:9] in cat14: return 14

In [63]:
base_path = '/home/edloftus/code/AtinDhawan/Dog_Identifier/raw_data/Attempt_2/All_Train'
train_dict = {}
for i in os.listdir(base_path):
    if i[:9] in all_cat:
        file_path = f'{base_path}/{i}'
        train_dict[file_path] = categorise(i)
len(train_dict)

1500

In [64]:
base_path = '/home/edloftus/code/AtinDhawan/Dog_Identifier/raw_data/Attempt_2/All_Test'
test_dict = {}
for i in os.listdir(base_path):
    if i[:9] in all_cat:
        file_path = f'{base_path}/{i}'
        test_dict[file_path] = categorise(i)
len(test_dict)

1133

In [65]:
X_train, y_train, X_test, y_test = list(train_dict.keys()), list(train_dict.values()), list(test_dict.keys()), list(test_dict.values())

In [49]:
def load_image(folder_name):
    image = tf.io.read_file(folder_name)
    image = tf.image.decode_jpeg(image, channels=3)
    return image

In [66]:
X_train = [load_image(i) for i in X_train]
X_test = [load_image(i) for i in X_test]

In [67]:
X_train, X_test = np.asarray(X_train), np.asarray(X_test)

In [73]:
y_train, y_test = np.asarray(y_train), np.asarray(y_test)

In [76]:
model = tf.keras.Sequential()

model.add(layers.Conv2D(16, kernel_size = (3,3), activation = 'relu', input_shape = (224,224,3)))
model.add(layers.MaxPool2D(pool_size=(2,2))) 

model.add(layers.Conv2D(32, kernel_size = (3,3), activation = 'relu'))
model.add(layers.MaxPool2D(pool_size=(2,2))) 

model.add(layers.Conv2D(64, kernel_size = (3,3), activation = 'relu'))
model.add(layers.MaxPool2D(pool_size=(2,2))) 

model.add(layers.Conv2D(128 ,kernel_size = (3,3) , padding = 'same' , activation='relu'))
model.add(layers.MaxPool2D(pool_size=(2,2)))     
          
model.add(layers.Flatten())
model.add(layers.Dense(15, activation = 'softmax'))

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_7 (Conv2D)           (None, 222, 222, 16)      448       
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 111, 111, 16)     0         
 2D)                                                             
                                                                 
 conv2d_8 (Conv2D)           (None, 109, 109, 32)      4640      
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 54, 54, 32)       0         
 2D)                                                             
                                                                 
 conv2d_9 (Conv2D)           (None, 52, 52, 64)        18496     
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 26, 26, 64)      

In [77]:
model.compile(loss = 'sparse_categorical_crossentropy',
              optimizer = optimizers.legacy.Adam(),
              metrics = ['accuracy'])

In [78]:
es = EarlyStopping(patience = 5)

model.fit(X_train, y_train, batch_size=32, epochs = 20, callbacks=[es], verbose = 1)

Epoch 1/20
47/47 [==============================] - 49s 1s/step - loss: 9.7048 - accuracy: 0.1093
Epoch 2/20
47/47 [==============================] - 49s 1s/step - loss: 2.2812 - accuracy: 0.2760
Epoch 3/20
47/47 [==============================] - 60s 1s/step - loss: 1.7965 - accuracy: 0.4133
Epoch 4/20
47/47 [==============================] - 48s 1s/step - loss: 1.2520 - accuracy: 0.6033
Epoch 5/20
47/47 [==============================] - 49s 1s/step - loss: 0.7838 - accuracy: 0.7540
Epoch 6/20
47/47 [==============================] - 48s 1s/step - loss: 0.4694 - accuracy: 0.8673
Epoch 7/20
47/47 [==============================] - 48s 1s/step - loss: 0.2309 - accuracy: 0.9333
Epoch 8/20
47/47 [==============================] - 50s 1s/step - loss: 0.1126 - accuracy: 0.9693
Epoch 9/20
47/47 [==============================] - 49s 1s/step - loss: 0.0847 - accuracy: 0.9787
Epoch 10/20
47/47 [==============================] - 49s 1s/step - loss: 0.0498 - accuracy: 0.9867
Epoch 11/20
47/47 [

In [79]:
model.evaluate(X_test, y_test)

36/36 [==============================] - 8s 224ms/step - loss: 7.6981 - accuracy: 0.2727


[7.69807243347168, 0.27272728085517883]